In [3]:
!pip -q install google-api-python-client google-auth-httplib2 google-auth-oauthlib

from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import default

SCOPES = ["https://www.googleapis.com/auth/documents"]

creds, _ = default(scopes=SCOPES)
docs_service = build("docs", "v1", credentials=creds)


In [4]:
MARKDOWN_NOTES = r"""# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [5]:
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple

@dataclass
class Block:
    kind: str  # heading / bullet / checkbox / paragraph / hr / footer
    text: str
    level: int = 0          # heading level (1/2/3) or bullet indent level
    meta: Optional[dict] = None  # extra info (e.g., mentions ranges)


MENTION_RE = re.compile(r"@\w+")

def _strip_parens_spaces(s: str) -> str:
    return s.strip()

def parse_markdown(md: str) -> List[Block]:
    """
    Minimal parser tuned for meeting-notes markdown:
    - Headings: #, ##, ###
    - Bullets: '-', '*', with nested indentation by leading spaces
    - Checkboxes: - [ ]
    - HR: ---
    - Footer: lines after HR
    """
    lines = md.splitlines()
    blocks: List[Block] = []
    after_hr = False

    for raw in lines:
        line = raw.rstrip("\n")
        if not line.strip():
            # Preserve blank lines as paragraph breaks (helps readability in Docs)
            blocks.append(Block(kind="paragraph", text=""))
            continue

        if line.strip() == "---":
            blocks.append(Block(kind="hr", text=""))
            after_hr = True
            continue

        if after_hr:
            # Everything after HR treat as footer
            blocks.append(Block(kind="footer", text=line.strip()))
            continue

        # Headings
        if line.startswith("# "):
            blocks.append(Block(kind="heading", text=line[2:].strip(), level=1))
            continue
        if line.startswith("## "):
            blocks.append(Block(kind="heading", text=line[3:].strip(), level=2))
            continue
        if line.startswith("### "):
            blocks.append(Block(kind="heading", text=line[4:].strip(), level=3))
            continue

        # Checkbox
        m_cb = re.match(r"^(\s*)-\s+\[\s*\]\s+(.*)$", line)
        if m_cb:
            indent_spaces = len(m_cb.group(1))
            level = indent_spaces // 2
            text = m_cb.group(2).strip()
            blocks.append(Block(kind="checkbox", text=text, level=level))
            continue

        # Bullets ( - or * )
        m_b = re.match(r"^(\s*)([-*])\s+(.*)$", line)
        if m_b:
            indent_spaces = len(m_b.group(1))
            level = indent_spaces // 2
            text = m_b.group(3).strip()
            blocks.append(Block(kind="bullet", text=text, level=level))
            continue

        # Plain paragraph
        blocks.append(Block(kind="paragraph", text=line.strip()))

    return blocks


In [6]:
def build_doc_text_and_ranges(blocks: List[Block]) -> Tuple[str, List[dict]]:
    """
    Create one big text payload + track paragraph ranges.
    Docs indices start at 1.
    Returns:
      full_text, paragraphs_metadata
    paragraphs_metadata items:
      {
        "kind": ..., "level": ..., "start": ..., "end": ...,
        "text": ..., "mentions": [(mstart, mend), ...]  # absolute doc indices
      }
    """
    full_lines = []
    meta = []
    cursor = 1  # Google Docs text starts at index 1

    def add_paragraph(text: str, kind: str, level: int):
        nonlocal cursor
        # Each paragraph ends with a newline for Docs
        line = text
        full_lines.append(line)

        start = cursor
        end = cursor + len(line) + 1  # include newline

        # Mentions absolute ranges
        mentions = []
        for mm in MENTION_RE.finditer(line):
            mentions.append((start + mm.start(), start + mm.end()))

        meta.append({
            "kind": kind,
            "level": level,
            "start": start,
            "end": end,
            "text": line,
            "mentions": mentions
        })

        cursor = end

    for b in blocks:
        if b.kind == "heading":
            add_paragraph(b.text, "heading", b.level)
        elif b.kind == "bullet":
            add_paragraph(b.text, "bullet", b.level)
        elif b.kind == "checkbox":
            add_paragraph(b.text, "checkbox", b.level)
        elif b.kind == "hr":
            # Simple horizontal divider line (Docs API also supports insertHorizontalRule,
            # but this keeps indexing easy in a single insertText)
            add_paragraph("—" * 24, "hr", 0)
        elif b.kind == "footer":
            add_paragraph(b.text, "footer", 0)
        else:
            # paragraph (including blank lines)
            add_paragraph(b.text, "paragraph", 0)

    full_text = "\n".join(full_lines) + "\n"  # final newline
    return full_text, meta


def build_formatting_requests(paragraphs_meta: List[dict]) -> List[dict]:
    requests = []

    # Helpers for paragraph style updates
    def set_heading_style(start, end, named_style_type: str):
        requests.append({
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {"namedStyleType": named_style_type},
                "fields": "namedStyleType"
            }
        })

    def set_indent(start, end, level: int):
        # Hanging indent looks good for bullets/checklists
        # 18pt per nesting level is a common readable default
        indent = 18 * (level + 1)  # base indent
        requests.append({
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {
                    "indentStart": {"magnitude": indent, "unit": "PT"},
                    "indentFirstLine": {"magnitude": -18, "unit": "PT"},
                },
                "fields": "indentStart,indentFirstLine"
            }
        })

    def apply_bullets(start, end, preset: str):
        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": preset
            }
        })

    def style_mentions(mentions: List[Tuple[int, int]]):
        for ms, me in mentions:
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": ms, "endIndex": me},
                    "textStyle": {
                        "bold": True,
                        "foregroundColor": {
                            "color": {"rgbColor": {"red": 0.11, "green": 0.45, "blue": 0.92}}
                        }
                    },
                    "fields": "bold,foregroundColor"
                }
            })

    def style_footer(start, end):
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "textStyle": {
                    "italic": True,
                    "fontSize": {"magnitude": 10, "unit": "PT"},
                    "foregroundColor": {
                        "color": {"rgbColor": {"red": 0.40, "green": 0.40, "blue": 0.40}}
                    }
                },
                "fields": "italic,fontSize,foregroundColor"
            }
        })

    # Pass 1: headings + mentions + footer style + bullet/checklist indentation & bullets
    for p in paragraphs_meta:
        kind, level, start, end = p["kind"], p["level"], p["start"], p["end"]

        if kind == "heading":
            if level == 1:
                set_heading_style(start, end, "HEADING_1")
            elif level == 2:
                set_heading_style(start, end, "HEADING_2")
            elif level == 3:
                set_heading_style(start, end, "HEADING_3")

        if p["mentions"]:
            style_mentions(p["mentions"])

        if kind == "footer":
            style_footer(start, end)

        if kind == "bullet":
            set_indent(start, end, level)
            # Standard bulleted list
            apply_bullets(start, end, "BULLET_DISC_CIRCLE_SQUARE")

        if kind == "checkbox":
            set_indent(start, end, level)
            # Real Google Docs checkbox bullets (works in Docs API)
            apply_bullets(start, end, "BULLET_CHECKBOX")

    return requests


In [7]:
def create_formatted_google_doc(markdown_text: str, title: str = "Product Team Sync") -> str:
    try:
        # 1) Create the doc
        doc = docs_service.documents().create(body={"title": title}).execute()
        doc_id = doc["documentId"]

        # 2) Parse markdown -> blocks
        blocks = parse_markdown(markdown_text)

        # 3) Build a single insertText payload + style ranges
        full_text, paragraphs_meta = build_doc_text_and_ranges(blocks)

        # Insert all text at index 1
        insert_req = [{
            "insertText": {
                "location": {"index": 1},
                "text": full_text
            }
        }]

        docs_service.documents().batchUpdate(
            documentId=doc_id,
            body={"requests": insert_req}
        ).execute()

        # 4) Apply formatting
        fmt_requests = build_formatting_requests(paragraphs_meta)

        try:
            docs_service.documents().batchUpdate(
                documentId=doc_id,
                body={"requests": fmt_requests}
            ).execute()
        except HttpError as e:
            # Fallback: if BULLET_CHECKBOX preset isn't supported in that environment,
            # re-run with a plain bullet preset for checkbox items.
            msg = str(e)
            if "BULLET_CHECKBOX" in msg or "bulletPreset" in msg:
                fmt_requests_fallback = []
                for r in fmt_requests:
                    if "createParagraphBullets" in r and r["createParagraphBullets"].get("bulletPreset") == "BULLET_CHECKBOX":
                        r = {
                            "createParagraphBullets": {
                                "range": r["createParagraphBullets"]["range"],
                                "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                            }
                        }
                    fmt_requests_fallback.append(r)

                docs_service.documents().batchUpdate(
                    documentId=doc_id,
                    body={"requests": fmt_requests_fallback}
                ).execute()
            else:
                raise

        return doc_id

    except HttpError as e:
        raise RuntimeError(f"Google Docs API error: {e}") from e
    except Exception as e:
        raise RuntimeError(f"Unexpected error: {e}") from e


doc_id = create_formatted_google_doc(
    MARKDOWN_NOTES,
    title="Product Team Sync - May 15, 2023"
)

print("Created doc:", f"https://docs.google.com/document/d/{doc_id}/edit")


Created doc: https://docs.google.com/document/d/1YyDz9sv7goSnDb5Q59rKa-nFtqVczxy_MkHYPa4YFIA/edit
